In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as mp
pCTR_train = pd.read_pickle('pCTR_validation')
rtb_validation = pd.read_csv('validation.csv')

In [99]:
# Make a table with the bidprice with premium only for impressions with  65%+ of click probability, otherwise bid 0.
# inear_bid = avgPayprice/1000 * (2*stDev)
df = pd.read_pickle('linear_bid')
df1 = df.drop(columns = ['bidprice'] )

df1['pCTR'].max()
#premium based on taking all impressions with click prob > 0.65
df1['premium'] = 50
df1['bidPrice'] = 0
df1['impression'] = 0
df1['winning'] = 0
#update bidPrice column with premium
df1.loc[df.pCTR>=0.65, 'bidPrice'] = df1['premium'] + df1['linear_bid']
#update impression column 
df1['impression'] = np.where ((df1['bidPrice'] >= df1['payprice']), 1, 0)
#df1['impression'].count()

#check the clicks
culo = df1['click'] == 1 
culo.groupby(df1['click']).count()
df1.head()
#df1.loc[df.impression == 1 & df.click == 1, 'bidPrice'] = df1['premium'] + df1['linear_bid']
#df1['pCTR'].max()
#df1['bidPrice'].mean()

,pCTR,linear_bid,click,bidid,payprice,premium,bidPrice,impression,winning
0,0.386555,81.996435,0,bbcb813b6166538503d8b33a5602d7d72f6019dc,23,50,0.0,0,0
1,0.386555,81.996435,0,5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d,75,50,0.0,0,0
2,0.386555,81.996435,0,f6ece71dae81d6b16bfb24ad6dd5611472d4c673,65,50,0.0,0,0
3,0.386555,81.996435,0,b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb,6,50,0.0,0,0
4,0.386555,81.996435,0,0899bf144249458ea9c89188473694bf44c7ca15,5,50,0.0,0,0


In [100]:
#Crop the df1 table to only take rows with an impression
df1_crop = df1[df1['impression'] == 1]
df1_crop.head()

,pCTR,linear_bid,click,bidid,payprice,premium,bidPrice,impression,winning
18,0.899329,190.766728,0,db12485ab1e09236b00aa8de2f19cac829711996,89,50,240.766728,1,0
47,0.899329,190.766728,0,3a37037477b74366a22b5697c5e71375efe880b0,35,50,240.766728,1,0
68,0.899329,190.766728,0,80b3fac4d107b44470d8f1452d69c367d73ca11a,100,50,240.766728,1,0
103,0.899329,190.766728,0,69907166a266e81bc8be86ef76ef9e630bdb9188,54,50,240.766728,1,0
106,0.899329,190.766728,0,c1e465ffd4900cf5ea02de878a2059624658a88a,16,50,240.766728,1,0


In [116]:
#run the algorithm to sepend the budget

budget = 6250
impression = 0
bids = 0
clicks = 0
for i in range(0, len(df1_crop)):
    if budget > 0 & budget > (df1_crop['payprice'].values[i]/1000):
        if (df1_crop['bidPrice'].values[i]) > (df1_crop['payprice'].values[i]):
            budget = budget - (df1_crop['payprice'].values[i]/1000)
            impressions += 1
            bids += 1
            clicks += df1_crop['click'].values[i] 
    else:
        print("Budget is spent!")
        break
consumedBudget = 6250 - budget
ctr = (clicks/bids)   
cpm = ((consumedBudget)/bids)*1000
cpc = (consumedBudget)/clicks 
#Output for the alrogithm
print("Output")
print("Number of Bids:",bids)
print("Number of Impressions:",impressions)
print("Number of Clicks:", clicks)
print("CTR is:",ctr, "%")
print("CPM is:",cpm)
print("CPC is:",cpc)

Budget is spent!


ZeroDivisionError: division by zero

In [2]:
#base bid should be equal to the equal to the median (=84) since this higher than the mode (=70) but lower than the mean (= 105). Using the mean won't be accurate because few outliers skew the mean to the right.
base_bid = 84/1000
avgCTR = 1792/2400000
pCTR = pCTR_train/1000
bidPrice = base_bid 

#lorenz linear bidding
budget = 6250
base = 0
bids = 0
impressions = 0
clicks = 0


for i in range(0,len(rtb_validation)):
    bidPrice = base_bid * (pCTR/avgCTR)
    if budget > bidPrice :
        if bidPrice > rtb_validation['bidprice'].values[i]/1000:
            bids += 1
            clicks += rtb_validation['click'].values[i]
            budget = budget - bidPrice
    else: 
        print("Budget is spent!")
        break
        
ctr = (clicks/bids)*100
cpm = 6250/bids
cpc = 6250/clicks



ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().